In [39]:
%load_ext autoreload
%autoreload 2

import torch
from torch.nn.functional import unfold, fold
from others.implementations import Convolution
from torch.nn.functional import conv2d

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Convolution

In [40]:
padding = 5
stride = 2
custom_conv = Convolution(3, 1, kernel_size=2, padding=padding, stride = stride)

inp = torch.ones((5,3,13,13))
inp[:,0,:,:] = 3
inp.requires_grad = True
out = custom_conv.forward(inp)
target = torch.zeros(out.size())

kernel = torch.ones((1,3,2,2))
kernel.requires_grad = True
kernel.retain_grad()
inp_padded = custom_conv.add_padding(inp, padding)
inp_padded = inp_padded.detach()
inp_padded.requires_grad = True
target_out = conv2d(inp_padded, kernel, stride=stride)
target_out.retain_grad()
loss = 0.5*((target-target_out)**2).sum()
loss.backward()
dl_dout = target_out.grad.detach()
dl_dw = kernel.grad.detach()
dl_dx = inp_padded.grad.detach()

In [41]:
#check out
(target_out - out).abs().sum()

tensor(0., grad_fn=<SumBackward0>)

In [42]:
#check dl_dx
(custom_conv.backward(dl_dout) - dl_dx).abs().sum()

tensor(0.)

In [43]:
#check dl_dw
(custom_conv.dl_dw - dl_dw).abs().sum()

tensor(0., grad_fn=<SumBackward0>)

## Up-sampling

In [399]:
from torch import nn
import others.implementations

In [400]:
scale_factor = 5
target_upsample = nn.Upsample(scale_factor = scale_factor, mode = 'nearest')
custom_upsample = others.implementations.Upsample(scale_factor = scale_factor)
inp = torch.ones((1,3,14,8))
inp.requires_grad = True
target_out = target_upsample.forward(inp)
target_out.retain_grad()
custom_out = custom_upsample.forward(inp)
target = torch.zeros((1,3,inp.shape[-2]*scale_factor, inp.shape[-1]*scale_factor))
loss = 0.5*((target-target_out)**2).sum()
loss.backward()


In [401]:
#check out
(target_out - custom_out).abs().sum()

tensor(0., grad_fn=<SumBackward0>)

In [402]:
#check dl_dx
(custom_upsample.backward(target_out)-inp.grad).abs().sum()

tensor(0., grad_fn=<SumBackward0>)

## Model

In [1]:
from others.implementations import *
import torch.nn as nn
%load_ext autoreload
%autoreload 2


In [2]:
inp_custom = torch.rand((1, 3, 32, 32))
inp_target = inp_custom.clone()
inp_target.requires_grad = True

In [3]:
custom_model = Sequential(
    Convolution(3, 5, kernel_size=2, stride=2, padding=0),
    ReLU(),
    Convolution(5, 5, kernel_size=2, stride=2, padding=0),
    ReLU(),
    Upsample(scale_factor=2),
    Convolution(5, 5, kernel_size=3, stride=1, padding=1),
    ReLU(),
    Upsample(scale_factor=2),
    Convolution(5, 1, kernel_size=3, stride=1, padding=1),
    Sigmoid()
    )


In [4]:
target_model = nn.Sequential(
    nn.Conv2d(3,5,kernel_size=2, stride=2, padding=0),
    nn.ReLU(),
    nn.Conv2d(5,5,kernel_size=2, stride=2, padding=0),
    nn.ReLU(),
    nn.Upsample(scale_factor=2),
    nn.Conv2d(5,5,kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.Upsample(scale_factor=2),
    nn.Conv2d(5,1,kernel_size=3, stride=1, padding=1),
    nn.Sigmoid()
    )
def init_weights(m):
    if isinstance(m, nn.Conv2d):
        nn.init.ones_(m.weight)
target_model = target_model.apply(init_weights)

In [5]:
#check out
custom_out = custom_model.forward(inp_custom)
target_out = target_model.forward(inp_target)
(custom_out - target_out).abs().sum()

c:\Users\stefa\deep-learning-project\Miniproject_2\others\implementations.py:76: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  return ((prev_dim - kernel_dim)//stride)+1


tensor(0., grad_fn=<SumBackward0>)

In [6]:
#check dl_dx
target = torch.zeros(custom_out.shape)
custom_mse = LossMSE()
target_mse = nn.MSELoss()
custom_loss = custom_mse.forward(custom_out, target)
target_loss =  target_mse(target_out, target)
inp_custom_grad = custom_mse.backward()
target_loss.backward()
(inp_custom_grad - inp_target.grad).abs().sum()

c:\Users\stefa\anaconda3\lib\site-packages\torch\autograd\__init__.py:173: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  C:\cb\pytorch_1000000000000\work\c10\cuda\CUDAFunctions.cpp:112.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


tensor(6144.)

In [7]:
from torch import optim
#check optimizer

custom_optimizer = optim_SGD(custom_model.param(), 0.01)
target_optimizer = optim.SGD(target_model.parameters(), lr=0.01)
custom_optimizer.step()
target_optimizer.step()

In [8]:
#check out
custom_out = custom_model.forward(inp_custom)
target_out = target_model.forward(inp_target)
(custom_out - target_out).abs().sum()

tensor(0., grad_fn=<SumBackward0>)